In [1]:
from owlready2 import Thing, get_ontology, AllDisjoint, sync_reasoner_pellet, Imp, FunctionalProperty
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto#")

# Define classes for the ontology based on the CSV file contents
with onto:
    class TVShow(Thing): pass
    class Episode(Thing): pass
    class User(Thing): pass
    class Comment(Thing): pass
    class Rating(Thing): pass
    class Emotion(Thing): pass

    # Define properties
    # Define relationships
    class posts(Comment >> User): pass
    class comments_on(Comment >> Episode): pass
    class follows(User >> TVShow): pass

    class has_watched(User >> Episode): pass
    class rated_by(Rating >> User): pass
    class rates(Rating >> Episode): pass
    class expressed_by(Emotion >> User): pass
    class evokes(Emotion >> Episode):  pass



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [13]:
import csv

def load_data(filename, process_row):
    with open(filename, 'rt', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=',')
        count = 0
        for row in reader:
            if count > 0 and count%100000 == 0:
                print(f'Processed {count} rows')
            process_row(row)
            count += 1

In [15]:
import datetime

def process_tv_shows_episodes(row):
    tv_show = onto.search_one(iri=f"{row['tv_show_name']}")
    if not tv_show:
        tv_show = TVShow(name=row['tv_show_name'])

    episode = onto.Episode(name=row['episode_id'])
    episode.aPourSaison = int(row['episode_season_number'])
    episode.aPourNumero = int(row['episode_number'])
    episode.rewatchedCount = int(row['rewatched_count'])

    episode.aPourDate = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()
    # episode.aPourDate = datetime.strptime(row['created_at'], '%Y-%m-%d')

path = './data/seen_episode_modified.csv'
load_data(path, process_tv_shows_episodes)
print(f'Loaded {len(onto.TVShow.instances())} TV shows and {len(onto.Episode.instances())} episodes')

Loaded 118 TV shows and 5239 episodes
